<a href="https://colab.research.google.com/github/aya-html/A-moroccan-XG-calculator/blob/main/unified_gmail_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 📥 Unified Gmail Assistant Notebook

**Purpose**: This notebook combines Phase 4 (Intelligence Layer) and Phase 5 (Autonomous Action Layer) into a unified AI assistant that processes Gmail threads, generates dual replies, classifies tone, scores confidence, and logs results to a Notion database.

---

### ✅ Modules Included:
- Gmail Fetch + Thread Mapping
- Summarization
- Command Detection
- Tone Classification
- Dual GPT-Based Replies
- Confidence Scoring
- Unified Notion Sync


In [ ]:
# === Gmail API Setup (Using Uploaded Token) ===
!pip install --quiet google-auth google-api-python-client

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
TOKEN_PATH = "token.json"

# Load the uploaded token
creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

# Connect to Gmail API
service = build('gmail', 'v1', credentials=creds)
print("✅ Gmail API is authenticated and ready.")


FileNotFoundError: [Errno 2] No such file or directory: 'token.json'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === Cell 2: Fetch Emails (Last 7 Days + Full Body + Language Detection) ===

from base64 import urlsafe_b64decode
from datetime import datetime, timedelta
import openai

def extract_text_from_parts(parts):
    for part in parts:
        if part.get("mimeType") == "text/plain":
            data = part["body"].get("data")
            if data:
                return urlsafe_b64decode(data).decode("utf-8", errors="ignore")
        elif "parts" in part:
            return extract_text_from_parts(part["parts"])
    return ""

def detect_language(text):
    prompt = f"What language is this email written in?\n\n{text[:500]}\n\nReply with only the language name."
    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return "[Error]"

# === Define time range (last 7 days) ===
now = datetime.utcnow()
past = (now - timedelta(days=7)).strftime('%Y/%m/%d')
query = f"after:{past} in:inbox"

fetched_emails = []

threads = service.users().threads().list(userId='me', q=query, maxResults=50).execute().get('threads', [])
print(f"📬 Found {len(threads)} threads...\n")

for thread in threads:
    try:
        msg = service.users().messages().get(userId='me', id=thread['id']).execute()
        payload = msg['payload']
        headers = {h['name']: h['value'] for h in payload['headers']}

        subject = headers.get("Subject", "(No Subject)")
        sender = headers.get("From", "")
        timestamp = int(msg.get("internalDate")) / 1000
        date_str = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        body = extract_text_from_parts(payload.get("parts", [])) or "[No text content]"
        language = detect_language(body)

        fetched_emails.append({
            "subject": subject,
            "sender": sender,
            "received_time": date_str,
            "body": body,
            "detected_language": language
        })
        print(f"✅ Processed: {subject} ({language})")
    except Exception as e:
        print(f"❌ Error processing thread: {e}")

# === Preview (Optional) ===
if fetched_emails:
    print("\n📧 Sample Preview:")
    print("Subject:", fetched_emails[0]["subject"])
    print("Language:", fetched_emails[0]["detected_language"])
    print("Body Preview:\n", fetched_emails[0]["body"][:1000])
else:
    print("⚠️ No valid emails found in inbox for the last 7 days.")


In [ ]:
# === Cell 3: Summarization Agent (Multilingual-Aware) ===

def summarize_email_multilingual(body, language):
    prompt = f"""You are a smart assistant. Summarize the email below in a clear, short paragraph (3–5 lines) for a team inbox dashboard.

Write the summary in this language: **{language}**

EMAIL:
---
{body}
---
SUMMARY:"""

    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=200
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return "[Summary Error]"

# Run summarization for each fetched email
for email in fetched_emails:
    body = email.get("body", "")
    language = email.get("detected_language", "English")
    summary = summarize_email_multilingual(body, language)
    email["summary"] = summary

# Optional: Show preview
print("✅ Summaries Generated:\n")
for i, email in enumerate(fetched_emails):
    print(f"📧 Email #{i+1}")
    print("Language:", email["detected_language"])
    print("Subject:", email["subject"])
    print("Summary:", email["summary"])
    print("=" * 80)


In [ ]:
!pip install --upgrade openai --quiet
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

# === Expanded Fixed Command List ===
COMMAND_LIST = [
    # Business / Ops
    "send_invoice", "billing_question", "pricing_request", "follow_up", "general_question",
    "account_closure", "update_contact", "change_account_details", "duplicate_request",
    # Marketing / CRM
    "unsubscribe", "feedback_positive", "complaint", "feature_request", "partnership_request",
    "event_registration", "customer_testimonial",
    # HR / Recruiting
    "job_application", "referral_submission", "interview_schedule_request", "cv_update_request",
    # Legal / Compliance
    "legal_inquiry", "contract_request", "privacy_policy_question", "data_deletion_request",
    # IT / Tech Support
    "technical_issue", "access_request", "reset_password", "security_alert", "bug_report", "system_down",
    # Logistics / Supply Chain
    "shipping_issue", "delivery_update_request", "return_request", "inventory_request",
    # Sales / Client Relations
    "schedule_demo", "confirm_availability", "send_proposal", "renew_contract", "custom_plan_request",
    # Docs / Resources
    "file_request", "request_report", "request_presentation", "send_agreement",
    # Meta
    "forward_to_support", "no_action"
]
COMMAND_LIST_STR = ", ".join(COMMAND_LIST)

# === GPT Classifier Function ===
def classify_multiple_commands(subject, summary, language="English"):
    prompt = f"""You are a multilingual business email classifier. Your task is to detect all valid commands (tasks or intents) from a business email based on the subject and summary provided.

COMMAND LIST:
{COMMAND_LIST_STR}

RULES:
- Output should be a Python list of command strings (lowercase) such as ["send_invoice", "follow_up"].
- Only include commands that are clearly mentioned or strongly implied.
- Do not invent commands outside the list.
- If no clear command exists, return only: ["no_action"]
- Language of the email: {language}

EMAIL SUBJECT:
"{subject}"

EMAIL SUMMARY:
"{summary}"

COMMANDS:"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        output = response.choices[0].message.content.strip()
        commands = eval(output) if output.startswith("[") else []
        return [cmd for cmd in commands if cmd in COMMAND_LIST]
    except Exception as e:
        return ["[Error]"]

# === Apply Classifier to All Emails ===
for email in fetched_emails:
    subject = email.get("subject", "")
    summary = email.get("summary", "")
    language = email.get("detected_language", "English")
    commands = classify_multiple_commands(subject, summary, language)
    email["detected_commands"] = commands

# === Preview Output ===
for i, email in enumerate(fetched_emails):
    print(f"📧 Email #{i+1}")
    print("Language:", email["detected_language"])
    print("Subject:", email["subject"])
    print("Commands:", email["detected_commands"])
    print("=" * 80)


In [ ]:
# === Cell 5: Thread Mapper – Link Messages to Gmail Thread IDs ===

for thread in threads:
    try:
        # Find the fetched email that matches the thread
        msg = service.users().messages().get(userId='me', id=thread['id']).execute()
        payload = msg.get("payload", {})
        headers = {h['name']: h['value'] for h in payload.get("headers", [])}
        subject = headers.get("Subject", "").strip()

        # Find by subject match
        for email in fetched_emails:
            if email["subject"].strip() == subject and not email.get("mapped_message_id"):
                email["mapped_message_id"] = msg["id"]
                break
    except Exception as e:
        print(f"❌ Error mapping thread: {e}")

# === Preview ===
print("✅ Thread Mapping Complete:\n")
for i, email in enumerate(fetched_emails):
    print(f"📧 Email #{i+1}")
    print("Subject:", email["subject"])
    print("Mapped Message ID:", email.get("mapped_message_id", "[Not Mapped]"))
    print("=" * 80)


In [ ]:
# === Cell 6: Replier Agent – Multilingual Dual Drafts ===

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

def generate_reply_drafts(subject, body, commands, language):
    if not commands or "no_action" in commands:
        return "[Skipped – Not a business-relevant message]", "[Skipped – Not a business-relevant message]"

    prompt_common = f"""
You are a professional email assistant representing a modern or multi-departmental company.
Your job is to write a helpful, human-readable, thoughtful, and accurate replies based on the user's original message.

You may be replying to:
- A customer or external partner (support, inquiries, feedback)
- A job applicant or contractor
- An internal employee (to HR, to CEO, to teammate)
- A request from management, a department, or executive team

Only generate a reply if the email appears to be:
✅ A real question, request, follow-up, or communication between people
✅ Related to company services, tasks, roles, schedules, reports, projects, hiring, policies

Do NOT generate a reply if:
❌ It's a system notification, marketing ad, newsletter, or automated alert
❌ It's from platforms like Google, Zoom, GitHub, LinkedIn, or Outlook auto-notifications
❌ The email is not clearly written by a person with intent

If the message is NOT appropriate for a human-written business reply, return exactly: [SKIP]


Context:
Language: {language}
Subject: {subject}
Commands: {', '.join(commands)}

Original Email:
---
{body}
---

Guidelines:
- Write in the same language as the email ({language}).
- Cover all user requests naturally (if more than one).
- Avoid copying the user’s email content.
- Be warm but professional. Clear and to-the-point.
- Keep each reply around 3–6 sentences.
- Close with a relevant polite sign-off (optional: your name or company).
"""

    # First version (Formal tone)
    prompt_v1 = prompt_common + "\nTone: Professional and clear.\n\nReply:"

    # Second version (Friendly + collaborative)
    prompt_v2 = prompt_common + "\nTone: Friendly and supportive.\n\nReply:"

    try:
        response_1 = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt_v1}],
            temperature=0.4,
            max_tokens=400
        )
        draft1 = response_1.choices[0].message.content.strip()

        response_2 = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt_v2}],
            temperature=0.6,
            max_tokens=400
        )
        draft2 = response_2.choices[0].message.content.strip()

        return draft1, draft2

    except Exception as e:
        error_msg = f"[Reply Error: {str(e)}]"
        return error_msg, error_msg

# === Generate Replies for All Emails ===
for email in fetched_emails:
    subject = email.get("subject", "")
    body = email.get("body", "")
    commands = email.get("detected_commands", [])
    language = email.get("detected_language", "English")

    draft1, draft2 = generate_reply_drafts(subject, body, commands, language)
    email["reply_draft_1"] = draft1
    email["reply_draft_2"] = draft2

# === Preview Drafts ===
print("✅ Reply Drafts Generated:\n")
for i, email in enumerate(fetched_emails, 1):
    print(f"📧 Email #{i}")
    print("Language:", email["detected_language"])
    print("Subject:", email["subject"])
    print("Commands:", ", ".join(email["detected_commands"]))
    print("Reply Draft 1:\n", email["reply_draft_1"])
    print("Reply Draft 2:\n", email["reply_draft_2"])
    print("=" * 80)


In [ ]:
# === Cell 7: Tone Detection + Confidence Score ===

def detect_tone(body, language="English"):
    prompt = f"""
You are a tone analysis expert.

Please classify the emotional tone of the following email in one word:
Choose only from: positive, neutral, negative, mixed

Language: {language}

EMAIL:
---
{body}
---

TONE:"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content.strip().lower()
    except:
        return "unknown"


def estimate_reply_confidence(draft, commands):
    if not draft or "[skip" in draft.lower() or "[reply error" in draft.lower():
        return 0
    if "no_action" in commands:
        return 20
    # Basic heuristic logic — can be replaced with GPT later
    score = 60
    if any(kw in draft.lower() for kw in ["let us know", "please find", "thank you", "i’ve forwarded", "attached", "we appreciate"]):
        score += 20
    if len(draft.split()) > 60:
        score += 10
    return min(score, 100)

# === Apply tone + confidence scoring to all emails
for email in fetched_emails:
    body = email.get("body", "")
    language = email.get("detected_language", "English")
    draft = email.get("reply_draft_1", "")
    commands = email.get("detected_commands", [])

    tone = detect_tone(body, language)
    confidence = estimate_reply_confidence(draft, commands)

    email["tone"] = tone
    email["reply_confidence"] = confidence

# === Preview
print("✅ Tone & Confidence Analysis:\n")
for i, email in enumerate(fetched_emails):
    print(f"📧 Email #{i+1}")
    print("Subject:", email["subject"])
    print("Tone:", email["tone"])
    print("Confidence Score:", email["reply_confidence"])
    print("=" * 60)


In [ ]:
# === Cell 8: Unified Notion Sync – Phase 4 + 5 Unified Logging ===

from notion_client import Client as NotionClient
from datetime import datetime

notion = NotionClient(auth=NOTION_TOKEN)
DATABASE_ID = NOTION_DB_ID

def determine_action_taken(email):
    if "[skip" in email.get("reply_draft_1", "").lower():
        return {"name": "Skipped", "color": "gray"}
    if "reply_error" in email.get("reply_draft_1", "").lower():
        return {"name": "Error", "color": "red"}
    return {"name": "Drafted Reply", "color": "orange"}

def determine_team_tag(commands):
    tags = []
    for cmd in commands:
        if cmd in ["schedule_demo", "send_proposal", "custom_plan_request"]:
            tags.append("Sales")
        elif cmd in ["technical_issue", "bug_report", "access_request"]:
            tags.append("Support")
        elif cmd in ["job_application", "cv_update_request", "hr_query"]:
            tags.append("HR")
        elif cmd in ["contract_request", "legal_inquiry"]:
            tags.append("Legal")
        elif cmd in ["shipping_issue", "inventory_request"]:
            tags.append("Ops")
    return list(set(tags)) or ["General"]

for email in fetched_emails:
    try:
        notion.pages.create(
            parent={"database_id": DATABASE_ID},
            properties={
                "Email Subject": {
                    "title": [{"text": {"content": email.get("subject", "(No Subject)")[:200]}}]
                },
                "Sender Email": {
                    "email": email.get("sender", "")
                },
                "Date": {
                    "date": {
                        "start": email.get("received_time", datetime.utcnow().isoformat())
                    }
                },
                "Summary": {
                    "rich_text": [{"text": {"content": email.get("summary", "")[:2000]}}]
                },
                "Detected Commands": {
                    "rich_text": [{"text": {"content": ", ".join(email.get("detected_commands", []))}}]
                },
                "Tone": {
                    "select": {"name": email.get("tone", "unknown")}
                },
                "Language": {
                    "rich_text": [{"text": {"content": email.get("detected_language", "unknown")}}]
                },
                "Reply Draft 1": {
                    "rich_text": [{"text": {"content": email.get("reply_draft_1", "")[:2000]}}]
                },
                "Reply Draft 2": {
                    "rich_text": [{"text": {"content": email.get("reply_draft_2", "")[:2000]}}]
                },
                "Confidence Score": {
                    "number": email.get("reply_confidence", 0)
                },
                "Action Taken": {
                    "select": determine_action_taken(email)
                },
                "Team Tag": {
                    "multi_select": [{"name": tag} for tag in determine_team_tag(email.get("detected_commands", []))]
                },
                "Fallback Used": {
                    "checkbox": "[reply error" in email.get("reply_draft_1", "").lower()
                },
                "Status": {
                    "select": {"name": "Pending"}
                }
            }
        )
        print(f"✅ Synced to Notion: {email['subject']}")
    except Exception as e:
        print(f"❌ Error syncing '{email.get('subject', '')}': {str(e)}")
